In [54]:
import os
import sys
# # sys.path.append("../eval/")
# sys.path.append("../eval/task")
os.chdir('..')
# sys.path = sys.path[:6]
# sys.path.append("/../")
print(sys.path)
import hashlib
import time
import random
import json
import openai
import transformers
import fnmatch
import datasets
import pathlib
import torch

from functools import cache
from collections import Counter
from datasets import load_dataset
from warnings import warn
from abc import abstractmethod, ABC
from eval import tasks
from eval.tasks import utils
from eval.utils import TokenizedDataset, complete_code
from eval.tasks.utils import evaluate, convert_to_nltk_rep
# from eval.generation import parallel_generations
from diskcache import Cache
from concurrent.futures import ThreadPoolExecutor


from torch.utils.data.dataloader import DataLoader
from transformers import StoppingCriteria, StoppingCriteriaList
from accelerate.utils import set_seed

from accelerate import Accelerator, DeepSpeedPlugin
from transformers import AutoModelForCausalLM, AutoTokenizer, HfArgumentParser

from eval import tasks
from eval.generation import parallel_generations
from eval.args import RunnerArguments, HFArguments, OAIArguments, GenerationArguments
# from eval.evaluator import HFEvaluator, OAIEvaluator
from eval.tasks import ALL_TASKS, TASK_REGISTRY

%load_ext autoreload
%reload_ext autoreload

transformers.logging.set_verbosity_error()
datasets.logging.set_verbosity_error()

['/home/gatum/Projects/Neurosymbolic-AI/linc2/linc_modify/notebooks', '/home/gatum/.conda/envs/linc_modify/lib/python310.zip', '/home/gatum/.conda/envs/linc_modify/lib/python3.10', '/home/gatum/.conda/envs/linc_modify/lib/python3.10/lib-dynload', '', '/home/gatum/.conda/envs/linc_modify/lib/python3.10/site-packages']
[2024-04-19 10:19:40,021] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [79]:
dataset = load_dataset('minimario/FOLIO')

In [80]:
train_fewshot_indices_all = [
    125,
    23,
    60,
    275,
    148,
    261,
    263,
    683,
    299,
    684,
    850,
    853,
    886,
    892,
    930,
    980,
]

In [81]:
def get_instructions(mode = 'neurosymbolic'):
    instructions = ""
    instructions += "The following is a first-order logic (FOL) problem.\n"
    instructions += "The problem is to determine whether the conclusion follows from the premises.\n"
    instructions += "The premises are given in the form of a set of first-order logic sentences.\n"
    instructions += "The conclusion is given in the form of a single first-order logic sentence.\n"
    if mode == "baseline":
        instructions += f"The task is to evaluate the conclusion as 'True', 'False', or 'Uncertain' given the premises."
    else:
        instructions += "The task is to translate each of the premises and conclusions into FOL expressions, "
        if mode == "scratchpad":
            instructions += f"and then to evaluate the conclusion as 'True', 'False', or 'Uncertain' given the premises."
        elif mode == "neurosymbolic":
            instructions += "so that the expressions can be evaluated by a theorem solver to determine whether the conclusion follows from the premises.\n"
            instructions += "Expressions should be adhere to the format of the Python NLTK package logic module."
    return instructions + "\n\n"

print(get_instructions())

The following is a first-order logic (FOL) problem.
The problem is to determine whether the conclusion follows from the premises.
The premises are given in the form of a set of first-order logic sentences.
The conclusion is given in the form of a single first-order logic sentence.
The task is to translate each of the premises and conclusions into FOL expressions, so that the expressions can be evaluated by a theorem solver to determine whether the conclusion follows from the premises.
Expressions should be adhere to the format of the Python NLTK package logic module.




In [82]:
def format_train_example(doc, mode='neurosymbolic'):
    example = format_test_example(doc)
    if mode == "baseline":
        example += f"{doc['label'].strip()}\n"
    elif mode == "cot":
        example += f"{doc['cot']}\n"
    else:
        for premise, fol in zip(doc["premises"], doc["premises-FOL"]):
            example += f"TEXT:\t{premise.strip()}\nFOL:\t{fol.strip()}\n"
        example += f"TEXT:\t{doc['conclusion'].strip()}\nFOL:\t{doc['conclusion-FOL'].strip()}\n"
        if mode == "scratchpad":
            example += f"ANSWER:\t{doc['label'].strip()}\n"
    return example + "</EVALUATE>\n"

In [83]:
def format_test_example(doc):
    example = "<PREMISES>\n"
    for premise in doc["premises"]:
        example += f"{premise.strip()}\n"
    example += "</PREMISES>\n"
    example += f"<CONCLUSION>\n{doc['conclusion'].strip()}\n</CONCLUSION>\n"
    example += "<EVALUATE>\n"
    return example

In [84]:
def reformat_fol_samples_train(train_dataset):
        def reformat_fol_sample(sample):
            sample["premises-FOL"] = [
                convert_to_nltk_rep(premise) for premise in sample["premises-FOL"]
            ]
            return sample

        return train_dataset.map(reformat_fol_sample)

def add_conclusion_fols_train(train_dataset):
    train_conclusion_fols = {
        23: "HigherRank(RealMadrid, Barcelona)",
        60: "-OlympicGoldMedalWinner(Amy) -> NobelLaureate(Amy)",
        125: "-Dispensable(Worksheet)",
        148: "FolkSong(Inception)",
        261: "MakeGoodBreakfast(Luke)",
        263: "exists x. (Develops(Ets, x) & For(x, k-OneTwoandhighereducation)) & exists x. (Develops(Ets, x) & AssociatedWith(x, Entrytouseducationinstitutions))",
        275: "ContributeToCountry(James)",
        299: "GetRhythmRight(John)",
        683: "exists x. (BRICS(x) & Speaks(x, Hindi))",
        684: "Film(Hamilton)",
        850: "-Liked(Leo, Charlie) & -Cares(Charlie, Leo)",
        853: "Won(Threebodyproblem, Hugoaward)",
        886: "Dagfinn(DagfinnAarskog)",
        892: "PartOf(Minsk, Scottishpremiership)",
        930: "-Locate(Boves, Europe)",
        980: "(InvitedTakePhoto(James) & -HappyCommunicate(James)) | (-InvitedTakePhoto(James) & HappyCommunicate(James))",
    }
    conclusions = [None for _ in range(len(train_dataset))]
    for index, conclusion_fol in train_conclusion_fols.items():
        conclusions[index] = conclusion_fol
    train_dataset = train_dataset.add_column("conclusion-FOL", conclusions)
    return train_dataset


In [99]:
def fewshot_examples():
    """
    Returns a few-shot example for the task.
    :param n: int
        number of examples
    :param seed: int
        seed for random number generator
    :return: str
    """
    
    _nshot = 1
    examples = []
    for doc in train.select(range(_nshot)):
        examples.append(format_train_example(doc))
    return "\n".join(examples)

In [97]:
# INIT
train_dataset = dataset['train']
_train_dataset = reformat_fol_samples_train(train_dataset)
_train_dataset = add_conclusion_fols_train(_train_dataset)
_train_dataset = _train_dataset.map(
    lambda x: {"label": "Uncertain" if x["label"] == "Unknown" else x["label"]},
    remove_columns=["label"],
)

shot = 1
train_fewshot_indices = train_fewshot_indices_all[:shot]
train = _train_dataset.select(train_fewshot_indices)
train

Dataset({
    features: ['conclusion', 'premises', 'premises-FOL', 'label', 'conclusion-FOL'],
    num_rows: 1
})

In [101]:
def get_prompt(doc):
    """
    Builds the prompt for the LM to generate from.
    :param doc: dict[str: str]
        sample from the test dataset
    :return: str
    """
    instructions = get_instructions()
    train = fewshot_examples()
    test = format_test_example(doc)
    prompt = "\n".join([instructions, train, test])
    return prompt

print(get_prompt(dataset['train'][0]))

The following is a first-order logic (FOL) problem.
The problem is to determine whether the conclusion follows from the premises.
The premises are given in the form of a set of first-order logic sentences.
The conclusion is given in the form of a single first-order logic sentence.
The task is to translate each of the premises and conclusions into FOL expressions, so that the expressions can be evaluated by a theorem solver to determine whether the conclusion follows from the premises.
Expressions should be adhere to the format of the Python NLTK package logic module.


<PREMISES>
All dispensable things are environment-friendly.
All woodware is dispensable.
All paper is woodware.
No good things are bad.
All environment-friendly things are good.
A worksheet is either paper or is environment-friendly.
</PREMISES>
<CONCLUSION>
A worksheet is not dispensable.
</CONCLUSION>
<EVALUATE>
TEXT:	All dispensable things are environment-friendly.
FOL:	all x. (Dispensable(x) -> EnvironmentFriendly(x)